In [ ]:
# ==============================================================================
# Autor(es): Juan Felipe Agudelo Rios                                         |
# Titulo: Cleaning                                                            |
# Fecha creación: 23/12/2023                                                  |
# Fecha última modificación: 26/12/2023                                       |
# ==============================================================================

In [8]:
import os
import pandas as pd
import numpy as np 
import regex as re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from ydata_profiling import ProfileReport
from sklearn.impute import KNNImputer

original_directory = "/Users/j.agudelo/Desktop/CasoFi/"

os.chdir(original_directory)

data = pd.read_excel("Input/AbandonoTC.xlsx")

data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022


In [9]:


CategoricalVars = ["Education_Level", "Income_Category", "Card_Category", "Marital_Status"]
TrasnformedVars=[]

def preprocess_data(data, categorical_vars):
    for var in categorical_vars:
        le = LabelEncoder()

        # Replace "Unknown" with NaN
        data = data.replace("Unknown", np.nan)

        # Create a mask for NaN values
        mask_nan = data[var].isnull()

        # Encode non-NaN values
        data.loc[~mask_nan, f"{var}_num"] = le.fit_transform(data.loc[~mask_nan, var])

        # Numerical variable
        Num_var = [
            'CLIENTNUM', 'Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count',
            'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
            'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1',
            'Avg_Utilization_Ratio',
            'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
            f'{var}_num'
        ]

        X = data[Num_var]

        # Impute missing values using KNNImputer
        imputer = KNNImputer(n_neighbors=3)
        X_imputed = imputer.fit_transform(X)
        data_imputed = pd.DataFrame(X_imputed, columns=Num_var)

        # Round and convert imputed values back to integers
        data[f'{var}_num'] = np.round(data_imputed[f'{var}_num']).astype(int)

        # Inverse transform for categorical variable
        data[var] = le.inverse_transform(data[f'{var}_num'].replace(le.classes_, 'Unknown'))

    return data

# Assuming you have a DataFrame named 'original_data'
data = preprocess_data(data, CategoricalVars)


encoder = OneHotEncoder(sparse=False) 

marital_status_reshaped = data["Marital_Status"].values.reshape(-1, 1)

encoded_vars = encoder.fit_transform(marital_status_reshaped)

encoded_df = pd.DataFrame(encoded_vars, columns=encoder.get_feature_names_out(["Marital_Status"]))
data = pd.concat([data, encoded_df], axis=1)

data["Attrition_Num"] = data["Attrition_Flag"].apply(lambda x: 0 if "Existing Customer" == str(x)  else 1)

data["Male"] = data["Gender"].apply(lambda x: 1 if str(x) == "M" else 0)



/Users/j.agudelo/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:

education_mapping = {
    'Uneducated': 0,
    'High School': 1,
    'Graduate': 2,
    'College': 3,
    'Post-Graduate': 4,
    'Doctorate': 5
}

data['Education_Level_num'] = data['Education_Level'].map(education_mapping)

income_mapping = {
    'Less than $40K': 0,
    '$40K - $60K': 1,
    '$60K - $80K': 2,
    '$80K - $120K': 3,
    '$120K +': 4
}

data['Income_Category_num'] = data['Income_Category'].map(income_mapping)

# Mapping for Card_Category
card_mapping = {
    'Blue': 0,
    'Silver': 3,
    'Gold': 1,
    'Platinum': 2
}

data['Card_Category_num'] = data['Card_Category'].map(card_mapping)

In [12]:
profile = ProfileReport(data, title="EDA Transformed DATA")
profile.to_file("Output/EDA/03_BaseTransFormada.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/j.agudelo/anaconda3/lib/python3.11/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Existing Customer'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
def clean_column_names(df):
    df.columns = (
        df.columns.str.strip()  # Remove leading/trailing whitespaces
        .str.lower()             # Convert to lowercase
        .str.replace(" ", "_")   # Replace spaces with underscores
        .str.replace(r"[^a-zA-Z0-9_]", "")  # Remove special characters
    )
    return df

data = clean_column_names(data)
data.to_excel("Output/DataFrames/CleanData.xlsx")
